In [1]:
def rule_to_binary(rule, num_states):
    return format(rule, '0' + str(num_states**3) + 'b')

def evolve_cell(left, center, right, rule_number):
    index = int(str(left)+str(center)+str(right), 2)
    if((1<<index)&rule_number)>0:
        return '1'
    return '0'

def evolve_ca(row, rule_numbers):
    num_cells = len(row)
    next_row =""

    for i in range(num_cells):
        if(i-1<0):
            left=0
        else:
            left = int(row[(i - 1) % num_cells])
        center = int(row[i])
        if(i+1==num_cells):
            right=0
        else:
            right = int(row[(i + 1) % num_cells])
        next_row+= evolve_cell(left, center, right, rule_numbers[i])
    return next_row

def binary_to_decimal(binary):
    decimal=int(binary,2)
    return decimal

def decimal_to_binary(number,n):
    binary=bin(number)
    binary=binary[2:]
    if len(binary) is not n:
        for i in range(n-len(binary)):
            binary='0'+binary
    return binary

def binatodeci(binary):
    return int(binary,2)
def generate_ca(number, rule_numbers, all_numbers,n):
    initial_row=decimal_to_binary(number,n)
    ca_history = [initial_row]
    #print("Intial configuration",initial_row)
    s=set()
    all_numbers[number]=1
    while True:
        new_row = evolve_ca(ca_history[-1], rule_numbers)
        #print("Next configuration",new_row)
        if new_row in s:
            return -1
        if new_row==initial_row:
            ca_history.append(new_row)
            break
        s.add(new_row)
        ca_history.append(new_row)
        # print(new_row)
        number=binatodeci(new_row)
        all_numbers[number]=1
    return ca_history

def get_index(arr):
    return arr[1]

In [2]:
def classification(data,testing_data,n_op,rule_numbers):
    #print("Cell Length:",n)
    t=2**n_op



    all_numbers=[0]*t
    ca_history=[]
    for i in range(t):
        if(all_numbers[i]==0):
            history = generate_ca(i, rule_numbers,all_numbers,n_op)
            #print("cycles generating:",history)
            ca_history.append(history)
            if history==-1:
                ca_history=[]
                break

    map_to_cycle={}

    #print(ca_history)
    #Number_of_cycles=len(ca_history)
    # print(Number_of_cycles)
    # Length of ca_history
    xor_values_for_each_cycle = []   # Stores [cycle_index, xor_value] for each cycle

    for i in range(len(ca_history)):   # Iterate over each cycle in the CA history
        xor_label_index_dict = []      # Temporary list to hold [cycle_index, xor_value]
        xor_a = 0

        for j in range(len(ca_history[i])):   # Iterate over each configuration in cycle i
            print (ca_history[i][j])
            num = binary_to_decimal(ca_history[i][j])   # Convert binary configuration to decimal
            xor_a = xor_a ^ num                         # Update XOR value with current
            map_to_cycle[ca_history[i][j]] = i          # Map configuration to its cycle index
        print("cycle index",i)
        print("xor value",xor_a)

        xor_label_index_dict.append(i)      # Add cycle index
        xor_label_index_dict.append(xor_a)  # Add final XOR value for this cycle

        xor_values_for_each_cycle.append(xor_label_index_dict)  # Store result

    # print(1)

    # First, find number of classes (maximum label + 1)
    num_classes = max(data.values()) + 1

    # Initialize count dictionary with zeroes for all classes
    count_majority_for_each_cycle = {}
    for i in range(len(ca_history)):
        count_majority_for_each_cycle[i] = [0] * num_classes
    print("Count_Majority in cycle",count_majority_for_each_cycle)

    # Count how many configs of each label fall in each cycle
    #print("data",data)

    #print("Cycle:",map_to_cycle)
    for config, label in data.items():
        # print("config and cycle config",map_to_cycle)
        cycle_index = map_to_cycle[config]
        count_majority_for_each_cycle[cycle_index][label] += 1
    print("Count_Majority in cycle",count_majority_for_each_cycle)
    #print("Rule:", rule_numbers)
    cycle_label={}
    print("\nAssigned cycle labels:")
    for cycle_id, label in cycle_label.items():
        print(f"Cycle {cycle_id}: Label = {label}")
    for i, (key, value) in enumerate(count_majority_for_each_cycle.items()):
        # i = index of the cycle
        # key = cycle id (e.g., 0, 1, 2, ...)
        # value = list of counts of class labels for this cycle

        label = value.index(max(value))
        # Find the label with the maximum count (majority label)

        c = 0
        for j in range(len(value)):
            if (value[label] == value[j]):
                c += 1
        # Count how many labels have the same max frequency (to detect ties)

        # Generate a unique conflict label ID
        conflict_label = max(
            set(label for label_list in count_majority_for_each_cycle.values()
                for label in range(len(label_list)))
        ) + 1
        # This ensures conflict labels are different from normal labels

        if c > 1:
            # If there is a tie (multiple labels with same max count)
            cycle_label[i] = conflict_label
            xor_values_for_each_cycle[i].append(conflict_label)
        else:
            # Otherwise, assign the majority label normally
            cycle_label[i] = label
            xor_values_for_each_cycle[i].append(label)

    for cycle_id, label in cycle_label.items():
        print(f"Cycle {cycle_id}: Label = {label}")
    # STEP 1: Sort based on some feature (XOR values )
    sorted_array = sorted(xor_values_for_each_cycle, key=get_index)
    #print("Rule:", rule_numbers)
    print("xor values",xor_values_for_each_cycle)
    print("sorted array",sorted_array)
    # STEP 2: Identify large gaps in index values to segment clusters
    gap_index_arr = [0]
    # Stores the indices where a "gap" is detected in sorted_array
    # Start with 0 (beginning of array)

    curr_diff = sorted_array[1][1] - sorted_array[0][1]
    # Initial difference between first two values (second column of sorted_array)

    for i in range(2, len(sorted_array)):
        diff = sorted_array[i][1] - sorted_array[i-1][1]
        # Compute difference between consecutive values

        if curr_diff == 0:
            curr_diff = diff
            # If current difference is 0, update it with new diff (to avoid divide-by-zero)

        elif diff > 10 * curr_diff:
            # If the new difference is much larger (10x threshold),
            # treat this as a "gap" → mark index i
            gap_index_arr.append(i)
            curr_diff = 0
            # Reset curr_diff so next comparison starts fresh
            continue

        curr_diff = diff
        # Update current difference for next iteration

    gap_index_arr.append(len(sorted_array))
# Always add the last index (end boundary)




    # STEP 3: Count labels in each cluster and resolve conflict (label == conflict_label)
    conflict_label = max([x[2] for x in sorted_array])  # assumed to be the last label (e.g., if k=4, conflict_label=4)
    num_classes = conflict_label  # excluding the conflict label

    j = 1
    label_count = [0] * num_classes  # dynamically sized
    for i in range(len(sorted_array) + 1):
        if i == gap_index_arr[j]:
            # determine the label with max frequency
            max_val = max(label_count)
            tied_labels = [i for i, count in enumerate(label_count) if count == max_val]

            # Check if tie
            if len(tied_labels) == 1:
                max_label = tied_labels[0]
            else:
                # Tie-break: use training data present in this cluster
                class_counts_in_cluster = {label: 0 for label in tied_labels}

                # Iterate through cycles in current cluster
                for k in range(gap_index_arr[j-1], gap_index_arr[j]):
                    cycle_id = sorted_array[k][0]
                    if cycle_id in count_majority_for_each_cycle:
                        for label in tied_labels:
                            class_counts_in_cluster[label] += count_majority_for_each_cycle[cycle_id][label]

                # Choose the label with highest count from training data
                max_label = max(class_counts_in_cluster, key=class_counts_in_cluster.get)

            # resolve conflicts in this cluster
            for k in range(gap_index_arr[j-1], gap_index_arr[j]):
                if sorted_array[k][2] == conflict_label:
                    cycle_id = sorted_array[k][0]
                    cycle_label[cycle_id] = max_label
                    sorted_array[k][2] = max_label
            # reset counters for next cluster
            label_count = [0] * num_classes
            j += 1

        if i < len(sorted_array):
            label = sorted_array[i][2]
            if label != conflict_label:
                label_count[label] += 1




    Cycle_representative_class_label = {}

    # pick first configuration from each cycle as representative
    for cycle_id, label in cycle_label.items():
        if cycle_id < len(ca_history) and ca_history[cycle_id]:
            representative_config = ca_history[cycle_id][0]  # first config in cycle
            Cycle_representative_class_label[representative_config] = label

    print("cycle reprentative and Cycle_representative_class_label:",representative_config, Cycle_representative_class_label)



    from sklearn.metrics import precision_score, recall_score, f1_score

# --- Testing ---
    count = 0
    y_true = []
    y_pred = []

    for i, (config, true_label) in enumerate(testing_data.items()):
        # Step 1: Split the configuration into n-sized chunks
        chunks = [config[j:j+n_op] for j in range(0, len(config), n_op)]
        chunk_labels = []

        for chunk in chunks:
            start_config = chunk
            current_config = chunk
            predicted = None

            while True:
                # evolve one step
                next_config = evolve_ca(current_config, rule_numbers)

                # check if next config has a representative label
                if next_config in Cycle_representative_class_label:
                    predicted = Cycle_representative_class_label[next_config]
                    break

                # stop if we came back to start (no representative found)  this will not happen since it is Reversible CA
                if next_config == start_config:
                    break

                # move forward
                current_config = next_config

            if predicted is not None:
                chunk_labels.append(predicted)

        # Step 2: Majority vote from chunk labels
        if chunk_labels:
            predicted_label = max(set(chunk_labels), key=chunk_labels.count)

            # Step 3: Compare with true label
            y_true.append(true_label)
            y_pred.append(predicted_label)

            if predicted_label == true_label:
                count += 1

    # --- Metrics ---
    accuracy = count / len(y_true) if y_true else 0.0
    precision = precision_score(y_true, y_pred, average="macro", zero_division=0)
    recall = recall_score(y_true, y_pred, average="macro", zero_division=0)
    f1 = f1_score(y_true, y_pred, average="macro", zero_division=0)

    # --- Results ---
    result = {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1_score": f1,
        "rule_numbers": rule_numbers.copy(),
        "Number_of_cycles": len(ca_history),
        "Cycle_lengths": [len(inner) - 1 for inner in ca_history]
    }

    return result


In [3]:
from sklearn.model_selection import KFold
import random
import time

from sklearn.model_selection import KFold
import random
import time
top_10_runs = []  # Will only ever store 10 best runs
# rule_numbers = []
max_ans=0
for times in range(1):
      initial_class = 0
      prev_rule = None
      new_training_data = {
    "00001": 0,
    "11110": 0,
    "01101": 0,
    "01110": 1,
    "11101": 1,
    "00110": 1
}


      testing_data = {
          "00001": 0,
           "00110": 1,
          "00101": 1
          # 00011# or 1, based on your ground truth
                    }

      # Generate rule sequence

      import pandas as pd

      n_opt=5

      rule_numbers=[3, 58, 195, 102, 17]
      print("Rule sequence:", rule_numbers)

      start_time = time.time()

      result = classification(new_training_data, testing_data, n_opt, rule_numbers)
      end_time = time.time()
      execution_time = end_time - start_time

      print(f"Execution time: {execution_time:.2f} seconds")
      print(f"Accuracy : {result['accuracy']:.4f}")
      print(f"Precision: {result['precision']:.4f}")
      print(f"Recall   : {result['recall']:.4f}")
      print(f"F1-Score : {result['f1_score']:.4f}")
      if(max_ans<result['accuracy']):
          max_ans=result['accuracy']

      print("Maximum Accuracy=",max_ans)


Rule sequence: [3, 58, 195, 102, 17]
00000
10101
01010
10011
01100
11101
00110
11011
00000
cycle index 0
xor value 0
00001
10110
01011
10000
01101
11110
00111
11000
00001
cycle index 1
xor value 1
00010
10111
01000
10001
01110
11111
00100
11001
00010
cycle index 2
xor value 2
00011
10100
01001
10010
01111
11100
00101
11010
00011
cycle index 3
xor value 3
Count_Majority in cycle {0: [0, 0], 1: [0, 0], 2: [0, 0], 3: [0, 0]}
Count_Majority in cycle {0: [0, 2], 1: [3, 0], 2: [0, 1], 3: [0, 0]}

Assigned cycle labels:
Cycle 0: Label = 1
Cycle 1: Label = 0
Cycle 2: Label = 1
Cycle 3: Label = 2
xor values [[0, 0, 1], [1, 1, 0], [2, 2, 1], [3, 3, 2]]
sorted array [[0, 0, 1], [1, 1, 0], [2, 2, 1], [3, 3, 2]]
cycle reprentative and Cycle_representative_class_label: 00011 {'00000': 1, '00001': 0, '00010': 1, '00011': 1}
Execution time: 0.01 seconds
Accuracy : 1.0000
Precision: 1.0000
Recall   : 1.0000
F1-Score : 1.0000
Maximum Accuracy= 1.0
